<a href="https://colab.research.google.com/github/MYinthewood/Seach-engine-TF_IDF-cos-similarity/blob/main/FAISS_Search_similar_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install faiss-gpu
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 12.5 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [9]:
# Verify that GPU is available
import torch
if torch.cuda.is_available():
    print('GPU is available.')
else:
    print('GPU is not available.')

GPU is available.


In [3]:
import faiss
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [32]:
df_pos = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FAISS/train_pos.csv")
df_pos.head()

,text
0,I am salivating for the sequels. I needed some...
1,The many other comments about the film say it ...
2,Now I love American Pie 1 and 2 while 3 was gr...
3,"A light, uplifting and engaging movie. Watchin..."
4,My husband and I enjoy The DoodleBops as much ...


In [33]:
len(df_pos)

12500

In [34]:
reviews = df_pos['text'].to_list()
# initialize sentence tSentenceTransformer model
model = SentenceTransformer("bert-base-nli-mean-tokens") #one vector per review
# create sentence embbeddings, one vecter per sentence sample like Doc2Vec
review_embeddings = model.encode(reviews)
review_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


(12500, 768)

In [35]:
# get dimension of the embedding single
d = review_embeddings.shape[1] # 768, vector length
nlist = 50 # ANN how many cell we want to split

In [37]:
# using the L2 index as a quantizer step
quantizer = faiss.IndexFlatL2(d)

#IVF reduces the scope to search，can use faiss.IndexIVFPQ to further speed up
index = faiss.IndexIVFFlat(quantizer, d, nlist)

# Train the index
index.train(review_embeddings)

# Add vectors to the index
index.add(review_embeddings) # add indices

In [39]:
# use reviews[10] as the query
query = reviews[10]
query_embedding = model.encode([reviews[10]]) # use reviews[10] as the query
print("Query is: ", query)

Query is:  Emilio Miraglio's "The Red Queen Kills Seven Times" (1972) is just about the most perfect example of a giallo that I have ever seen, mixing all the requisite elements into one sinister stew indeed. First of all, and of paramount importance for me, it has a complex, twisty plot that ultimately makes perfect sense, and the killer here does not come completely out of left field at the end. The story, concerning a series of gruesome murders (you already know how many from the film's title, right?) that takes place in seeming fulfillment of an ancient prophecy concerning two sisters, is an involving one, and the murderer, a red-cloaked figure with the insane laugh of a madwoman, is both frightening and memorable. Every great giallo requires some lovely lead actresses, and here we have quite an assortment, headed by the ridiculously beautiful Barbara Bouchet as one of the two sisters and, in one of her earlier roles, Sybil Danning, as a lustful tramp at Barbara's fashion house. An

In [ ]:
k = 6 # search for top 6 similar reviews, but the first one would be the query itself
xq = model.encode([reviews[10]]) # use reviews[10] as the query
print(reviews[10])
D, I = index.search(xq, k)

In [45]:
# Perform a k-nearest neighbor search
k = 6  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding.reshape(1, -1), k)

# Print the most similar reviews
print(f"Query: {query}", "\n")
for i, idx in enumerate(indices[0]):
    print(f"Similar review {i+1}: {reviews[idx]}")
    print(f"Distance: {distances[0][i]}")
    print("\n")
#距离越小越相似

Query: Emilio Miraglio's "The Red Queen Kills Seven Times" (1972) is just about the most perfect example of a giallo that I have ever seen, mixing all the requisite elements into one sinister stew indeed. First of all, and of paramount importance for me, it has a complex, twisty plot that ultimately makes perfect sense, and the killer here does not come completely out of left field at the end. The story, concerning a series of gruesome murders (you already know how many from the film's title, right?) that takes place in seeming fulfillment of an ancient prophecy concerning two sisters, is an involving one, and the murderer, a red-cloaked figure with the insane laugh of a madwoman, is both frightening and memorable. Every great giallo requires some lovely lead actresses, and here we have quite an assortment, headed by the ridiculously beautiful Barbara Bouchet as one of the two sisters and, in one of her earlier roles, Sybil Danning, as a lustful tramp at Barbara's fashion house. Anothe

In [28]:
# 一个例子

In [46]:
# Example data
data = {'text': ['I am so happy.', 'Today is raining.', 'He is a school teacher.']}
df_pos = pd.DataFrame(data)

# Extract reviews from DataFrame
reviews = df_pos['text'].tolist()

# Initialize SentenceTransformer model
model = SentenceTransformer("bert-base-nli-mean-tokens")

# Create sentence embeddings
review_embeddings = model.encode(reviews)

# Shape of embeddings
print("Shape of review embeddings:", review_embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Shape of review embeddings: (3, 768)


In [58]:
# Setup Faiss for similarity search
d = review_embeddings.shape[1]
nlist = 3  # Number of clusters
quantizer = faiss.IndexFlatL2(d)  # L2 distance quantizer
index = faiss.IndexIVFFlat(quantizer, d, nlist)  # IVF index

# Optional: Use Product Quantization for even more speed-up (commented out)
# M = 8  # Number of subquantizers
# nbits = 8  # Number of bits per subvector
# index = faiss.IndexIVFPQ(quantizer, d, nlist, M, nbits)

# Train the index
index.train(review_embeddings)

# Add vectors to the index
index.add(review_embeddings)

In [59]:
# Example query to find similar reviews
query = "My day is good."
query_embedding = model.encode([query])[0]

In [60]:
print(len(query_embedding))
print(query_embedding.shape)

768
(768,)


In [61]:

print(len(query_embedding.reshape(1, -1)))
print(query_embedding.reshape(1, -1).shape)

1
(1, 768)


In [62]:
# Perform a k-nearest neighbor search
k = 2  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding.reshape(1, -1), k)

# Print the most similar reviews
print(f"Query: {query}")
for i, idx in enumerate(indices[0]):
    print(f"Similar review {i+1}: {reviews[idx]}")
    print(f"Distance: {distances[0][i]}")
    print('\n')

Query: My day is good.
Similar review 1: I am so happy.
Distance: 90.68080139160156


Similar review 2: He is a school teacher.
Distance: 3.4028234663852886e+38


